#Experment-1:A Multilayer Perceptron (MLP) using Keras with TensorFlow, and finetune neural network hyper parameters for regression problem

Aim: To implement a Multilayer Perceptron (MLP) using Keras with TensorFlow, and finetune neural network hyper parameters for regression problem (house price prediction).

In [ ]:
pip install -q -U keras-tuner

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from kerastuner.tuners import Hyperband

# Load the California housing dataset
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['price'] = data.target

# Split the data into features and target
X = df.drop('price', axis=1)
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model building function
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', 32, 256, step=32),
                    activation='relu', input_dim=X_train_scaled.shape[1]))
    model.add(Dense(units=hp.Int('units_2', 32, 128, step=32),
                    activation='relu'))
    model.add(Dense(units=hp.Int('units_3', 32, 64, step=32),
                    activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Instantiate the Hyperband tuner
tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=2,
    factor=2,
    directory='my_dir',
    project_name='regression_tuner'
)

# Early stopping callback to monitor validation loss and stop training if no improvement after 3 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Search for the best hyperparameters and get the best model
tuner.search(X_train_scaled, y_train, epochs=5, validation_split=0.2)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])

# Displaying the summary of the model
model.summary()

# Evaluate the best model on the test set
mse = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f'Mean Squared Error on Test Set: {mse:.4f}')

In [ ]:
#@title Run the code to to remove my_dir
import shutil

# Replace 'your_directory' with the actual path of the directory you want to remove
directory_to_remove = 'my_dir'

# Remove the directory recursively
shutil.rmtree(directory_to_remove)

print(f"Directory '{directory_to_remove}' removed.")


run the above if you want to restart the hyperparm tunning. else it take already compiled best model

# code explanation

In [ ]:
#@title Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from kerastuner.tuners import Hyperband

In [ ]:
#@title Load the California housing dataset
# Load the California housing dataset using sklearn's fetch_california_housing function
data = fetch_california_housing()

# Create a DataFrame (df) using the dataset features as columns and add the target variable ('price')
df = pd.DataFrame(data.data, columns=data.feature_names)
df['price'] = data.target

**fetch_california_housing function from sklearn**: It then creates a pandas DataFrame (df) using the dataset's feature values as columns and adds the target variable ('price') to the DataFrame. The resulting DataFrame contains both the input features and the target variable for further analysis or modeling.

In [ ]:
#@title Split the data into features and target
# Split the data into features (X) and target variable (y)
X = df.drop('price', axis=1)  # X contains all columns except 'price'
y = df['price']  # y contains only the 'price' column

* **X**: This variable holds the features (independent variables) and is created by dropping the 'price' column using df.drop('price', axis=1). It includes all columns from the original DataFrame except for the 'price' column.

* **y**: This variable represents the target variable (dependent variable) and is created by extracting the 'price' column from the original DataFrame using df['price']. It contains the values we want to predict or model.

In [ ]:
#@title Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In this code snippet, the dataset is split into training and testing sets using the `train_test_split` function from scikit-learn. Here's a breakdown of the parameters:

- `X`: The feature dataset.
- `y`: The target variable.
- `test_size=0.2`: Specifies that 20% of the data will be used for testing, and the remaining 80% will be used for training.
- `random_state=42`: Provides a seed for reproducibility, ensuring that the same split is obtained each time the code is executed.

The result is four sets:

- `X_train`: The training set features.
- `X_test`: The testing set features.
- `y_train`: The training set target variable.
- `y_test`: The testing set target variable.

This separation allows for training a model on one subset of the data (training set) and evaluating its performance on another subset (testing set), helping to assess the model's generalization to new, unseen data.

In [ ]:
#@title Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In this code snippet, the input features (`X_train` and `X_test`) are standardized using the `StandardScaler` from scikit-learn. Standardization is a common preprocessing step in machine learning that transforms the data to have a mean of 0 and a standard deviation of 1. This is achieved by subtracting the mean and dividing by the standard deviation.

Here's a breakdown of the code:

- `scaler`: An instance of the `StandardScaler` class.
- `fit_transform(X_train)`: Fits the scaler to the training data (`X_train`) and transforms it. The `fit_transform` method is used for training the scaler and applying the transformation simultaneously.
- `transform(X_test)`: Applies the same transformation to the testing data (`X_test`). It's important to use the same parameters (mean and standard deviation) obtained from the training data to standardize the testing data.

Standardization helps ensure that the features are on a similar scale, preventing one feature from dominating others during model training. This is especially important for algorithms sensitive to the scale of input features, such as gradient-based optimization algorithms used in neural networks.

In [ ]:
#@title Define the model building function
def build_model(hp):
    """
    Function to build a regression model with hyperparameters to be tuned.

    Parameters:
    - hp (HyperParameters): Hyperparameter tuning object.

    Returns:
    - model (Sequential): Compiled Keras model.
    """
    model = Sequential()

    # Add the input layer with units sampled from the specified range
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=256, step=32),
                    activation='relu', input_dim=X_train_scaled.shape[1]))

    # Add the first hidden layer with units sampled from the specified range
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=128, step=32),
                    activation='relu'))

    # Add the second hidden layer with units sampled from the specified range
    model.add(Dense(units=hp.Int('units_3', min_value=32, max_value=64, step=32),
                    activation='relu'))

    # Add the output layer with linear activation for regression
    model.add(Dense(1, activation='linear'))

    # Compile the model with Adam optimizer and mean squared error loss
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model


**Explanation:**

- `hp.Int`: This function is used to sample integers from a specified range. In this case, it's used to determine the number of units in each layer.
- `Sequential`: The Keras sequential model is used to linearly stack layers.
- `Dense`: This function adds a fully connected layer to the model.
- `input_dim`: Specifies the input size for the first layer based on the number of features in the training data.
- `activation='relu'`: Rectified Linear Unit (ReLU) activation is used for hidden layers.
- `activation='linear'`: Linear activation is used for the output layer in regression problems.
- `optimizer='adam'`: Adam optimizer is used for optimization.
- `loss='mean_squared_error'`: Mean squared error is used as the loss function for regression.

**Why should I have to use above function?**
- The `build_model(hp)` function is designed for hyperparameter tuning using Keras Tuner. Keras Tuner is a library that helps you optimize hyperparameters for your machine learning models. In the context of the code you provided, here are the reasons for using `build_model(hp)`:

1. **Hyperparameter Tuning**: The purpose of this function is to define the architecture of your neural network model, where certain architectural hyperparameters are dynamically set using the `hp` object from Keras Tuner. These hyperparameters include the number of units in each hidden layer.

2. **Flexibility for Search Space**: By using `hp` functions such as `hp.Int`, you define a search space for hyperparameters. This allows Keras Tuner to search for the optimal combination of hyperparameters within the specified ranges during the tuning process.

3. **Automated Search**: Keras Tuner performs an automated search over the hyperparameter space defined in `build_model(hp)` to find the combination that minimizes the specified objective (e.g., validation loss). This automates the process of finding the best hyperparameters, saving you from manually trying different combinations.

4. **Reproducibility**: By encapsulating the model creation within a function, you ensure that each configuration is created in a consistent manner. This is crucial for reproducibility, especially when trying to reproduce experiments or share your work with others.

5. **Integration with Keras Tuner**: The `hp` object provides a convenient way to sample hyperparameters, and it integrates seamlessly with Keras Tuner's search algorithms. It allows Keras Tuner to explore the hyperparameter space efficiently and identify the optimal configuration for your model.

In summary, using `build_model(hp)` is a key step when leveraging Keras Tuner for hyperparameter tuning. It defines the search space and allows the tuner to explore different configurations to find the best set of hyperparameters for your regression model.

In [ ]:
#@title Instantiate the Hyperband tuner
tuner = Hyperband(
    build_model,
    objective='val_loss',  # Objective to minimize (validation loss in this case)
    max_epochs=2,  # Maximum number of epochs for each trial
    factor=2,  # Reduction factor for the number of trials in each bracket
    directory='my_dir',  # Directory to store tuning results
    project_name='regression_tuner'  # Name of the tuning project
)


**Explanation:**

- **Hyperband Tuner**: The Hyperband tuner is used for hyperparameter tuning. It is a bandit-based optimization algorithm that efficiently explores the hyperparameter space by allocating resources to different configurations. Hyperband performs successive halving with early-stopping and aims to find the best set of hyperparameters for a given machine learning model.

- **build_model**: The `build_model` function is a model-building function that defines the architecture of the neural network. It takes hyperparameters as input and returns a compiled Keras model.

- **Objective ('val_loss')**: The objective of the tuning is to minimize the validation loss. During the tuning process, the tuner explores different sets of hyperparameters and evaluates their performance based on the validation loss.

- **Max Epochs (2)**: The maximum number of epochs to train the model for each trial. This is a hyperparameter for the tuner, specifying the maximum duration of each trial.

- **Factor (2)**: The reduction factor for the number of trials in each bracket. Hyperband uses successive halving, and the factor determines how many configurations are promoted to the next round. A higher factor explores more configurations but requires more resources.

- **Directory ('my_dir')**: The directory where the tuning results are stored. It helps in resuming interrupted tuning processes or retrieving information about completed trials.

- **Project Name ('regression_tuner')**: The name of the tuning project. It is used to organize and distinguish different tuning projects when using the same directory.

In summary, the Hyperband tuner efficiently searches the hyperparameter space to find the best configuration for the neural network model, aiming to minimize the validation loss. The tuner automates the process of hyperparameter tuning, making it easier to find optimal settings for improved model performance.

In [ ]:
#@title Early stopping callback to monitor validation loss and stop training if no improvement after 3 epochs
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Metric to monitor for improvement
    patience=3  # Number of epochs with no improvement after which training will be stopped
)

**Explanation:**

- `monitor='val_loss'`: The metric to monitor for improvement, which is the validation loss in this case.
- `patience=3`: The number of epochs with no improvement after which training will be stopped. If the validation loss does not improve for 3 consecutive epochs, training will be halted early.

In [ ]:
#@title Search for the best hyperparameters and get the best model
tuner.search(
    X_train_scaled,  # Scaled training features
    y_train,  # Training target variable
    epochs=5,  # Number of training epochs
    validation_split=0.2  # Fraction of the training data to use as validation set
)

**Explanation:**

- `X_train_scaled`: The scaled training features used for training the model.
- `y_train`: The training target variable.
- `epochs=5`: The number of training epochs, i.e., the number of times the entire training dataset is passed forward and backward through the neural network.
- `validation_split=0.2`: Fraction of the training data to use as a validation set. The model's performance on the validation set is monitored during training, and the hyperparameter search is guided by this validation performance.

In [ ]:
#@title Get the best hyperparameters, build the model, and train with early stopping
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    X_train_scaled,  # Scaled training features
    y_train,  # Training target variable
    epochs=10,  # Number of training epochs
    validation_split=0.2,  # Fraction of the training data to use as validation set
    callbacks=[stop_early]  # Early stopping callback
)

**Explanation:**

- `best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]`: Retrieves the best hyperparameters found during the search.
- `model = tuner.hypermodel.build(best_hps)`: Builds a new model using the best hyperparameters.
- `history = model.fit(...)`: Trains the model with the scaled training features (`X_train_scaled`) and training target variable (`y_train`). The training process is monitored using the validation set, and early stopping is applied with the specified callback (`stop_early`).

In [ ]:
#@title Display the summary of the trained model
model.summary()

**Explanation:**

- `model.summary()`: Displays a summary of the architecture and parameters of the trained neural network model. This includes information about each layer, the number of parameters, and the output shape at each layer. The summary provides a concise overview of the model's structure.

In [ ]:
#@title Evaluate the best model on the test set
mse = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f'Mean Squared Error on Test Set: {mse:.4f}')

**Explanation:**

- `model.evaluate(X_test_scaled, y_test, verbose=0)`: Evaluates the trained model on the test set using the specified input features (`X_test_scaled`) and target values (`y_test`). The `verbose=0` parameter ensures that the evaluation is performed silently without displaying progress information.
- `print(f'Mean Squared Error on Test Set: {mse:.4f}')`: Prints the mean squared error (MSE) on the test set. The `f-string` is used to format the output and display the MSE with four decimal places.